# Update Oxford Nanopore tables

This notebook updates the 'sample' table of available ONT flowcells, 'sample_set' table (one or more flowcells per participant), and 'participant' table (list of unique individuals discovered).

To auto-populate these tables, this notebook scans files in the gs://broad-gp-oxfordnano and gs://broad-dsde-methods-long-reads-deepseq buckets and extracts relevant metadata from the final_summary.txt files.

If changes were made to the 'sample' table in Terra, we take care not to overwrite those changes. If one wishes to restore those entries to their original values, the rows should first be deleted from the Terra table.

All other tables are auto-generated based on the 'sample' table.

## Setup

Install and import some packages that we're going to need.  Set up some helper functions.

In [1]:
#!pip install --use-feature=2020-resolver --upgrade pip pandas_gbq google-cloud-storage google-cloud-bigquery fastnumbers xmltodict firecloud > /dev/null 2>/dev/null

In [2]:
import os
import re
import hashlib

import pandas as pd
import firecloud.api as fapi

from google.cloud import bigquery
from google.cloud import storage
from google.api_core.exceptions import NotFound

from collections import OrderedDict

import xmltodict
import pprint

In [26]:
def load_summaries(gcs_buckets):
    storage_client = storage.Client()
    schemas = OrderedDict()

    ts = []
    for gcs_bucket in gcs_buckets:
        blobs = storage_client.list_blobs(re.sub("^gs://", "", gcs_bucket))

        for blob in blobs:
            if 'final_summary' in blob.name:
                doc = blob.download_as_string()
                t = {}
                
                for line in doc.decode("utf-8").split("\n"):
                    if '=' in line:
                        k,v = line.split('=')
                    
                        t[k] = v
                        
                t['Files'] = {
                    'final_summary.txt': gcs_bucket + "/" + blob.name,
                    'sequencing_summary.txt': 'missing'
                }

                bs = storage_client.list_blobs(re.sub("^gs://", "", gcs_bucket), prefix=os.path.dirname(blob.name) + "/" + t['sequencing_summary_file'])
                for b in bs:
                    t['Files']['sequencing_summary.txt'] = gcs_bucket + "/" + b.name
                    
                if 'sequencing_summary.txt' not in t['Files']:
                    pp = pprint.PrettyPrinter(indent=4)
                    pp.pprint(t)

                ts.append(t)

    return ts


def upload_data(namespace, workspace, tbl):
    # delete old sample set
    ss_old = fapi.get_entities(namespace, workspace, f'sample_set').json()
    sample_sets = list(map(lambda e: e['name'], ss_old))
    f = [fapi.delete_sample_set(namespace, workspace, sample_set_index) for sample_set_index in sample_sets]
    
    # delete old samples
    s_old = fapi.get_entities(namespace, workspace, 'sample').json()
    samples = list(map(lambda e: e['name'], s_old))
    f = [fapi.delete_sample(namespace, workspace, sample_index) for sample_index in samples]

    # upload new samples
    a = fapi.upload_entities(namespace, workspace, entity_data=tbl.to_csv(index=False, sep="\t"), model='flexible')

    if a.status_code == 200:
        print(f'Uploaded {len(tbl)} rows successfully.')
    else:
        print(a.json())

    # upload new sample set
    ss = tbl.filter(['sample_name'], axis=1).drop_duplicates()
    ss.columns = [f'entity:sample_set_id']
    
    b = fapi.upload_entities(namespace, workspace, entity_data=ss.to_csv(index=False, sep="\t"), model='flexible')
    if b.status_code == 200:
        print(f'Uploaded {len(ss)} sample sets successfully.')
    else:
        print(b.json())
    
    # upload membership set
    ms = tbl.filter(['sample_name', 'entity:sample_id'], axis=1).drop_duplicates()
    ms.columns = [f'membership:sample_set_id', f'sample']
    
    c = fapi.upload_entities(namespace, workspace, entity_data=ms.to_csv(index=False, sep="\t"), model='flexible')
    if c.status_code == 200:
        print(f'Uploaded {len(ms)} sample set members successfully.')
    else:
        print(c.json())


## Environment

Set up our environment (Terra namespace, workspace, and the location of ONT bucket(s)).

In [4]:
namespace = os.environ['GOOGLE_PROJECT']
workspace = os.environ['WORKSPACE_NAME']
default_bucket = os.environ['WORKSPACE_BUCKET']

gcs_buckets_ont = ['gs://broad-gp-oxfordnano', 'gs://broad-dsde-methods-long-reads-deepseq']

In [5]:
print(namespace)
print(workspace)

broad-firecloud-dsde-methods
dsp-oxfordnano


## Retrieve existing sample table from Terra

If it exists, retrieve the 'sample' table from this workspace.

In [6]:
ent_old = fapi.get_entities(namespace, workspace, 'sample').json()

if len(ent_old) > 0:
    tbl_old = pd.DataFrame(list(map(lambda e: e['attributes'], ent_old)))
    tbl_old["entity:sample_id"] = list(map(lambda f: hashlib.md5(f.encode("utf-8")).hexdigest(), tbl_old["final_summary_file"]))

## Examine Oxford Nanopore final_summary.txt files

Navigate GCS directories and look for ONT flowcells (indicated by the presence of the final_summary.txt files).

In [7]:
ts = load_summaries(gcs_buckets_ont)

In [8]:
tbl_header = ["final_summary_file", "sequencing_summary_file", "protocol_group_id", "instrument", "position", "flow_cell_id", "sample_name", "basecalling_enabled", "started", "acquisition_stopped", "processing_stopped", "fast5_files_in_fallback", "fast5_files_in_final_dest", "fastq_files_in_fallback", "fastq_files_in_final_dest"]
tbl_rows = []

for e in ts:
    tbl_rows.append([
        e["Files"]["final_summary.txt"],
        e["Files"]["sequencing_summary.txt"],
        e["protocol_group_id"],
        e["instrument"],
        e["position"],
        e["flow_cell_id"],
        e["sample_id"],
        e["basecalling_enabled"],
        e["started"],
        e["acquisition_stopped"],
        e["processing_stopped"],
        e["fast5_files_in_fallback"],
        e["fast5_files_in_final_dest"],
        e["fastq_files_in_fallback"],
        e["fastq_files_in_final_dest"]
    ])

tbl_new = pd.DataFrame(tbl_rows, columns=tbl_header)
tbl_new["entity:sample_id"] = list(map(lambda f: hashlib.md5(f.encode("utf-8")).hexdigest(), tbl_new["final_summary_file"]))

## Merge old and new sample list

If there are changes to the old sample list, make sure we retain them through subsequent table updates.  Do not overwrite old sample entries (metadata may have been manually modified).

In [9]:
if len(ent_old) > 0:
    for sample_id in tbl_old.merge(tbl_new, how='outer', indicator=True).loc[lambda x : x['_merge']=='left_only']['entity:sample_id'].tolist():
        print(f'Entry for sample {sample_id} has been modified.  Keeping changes.')
        tbl_new = tbl_new[tbl_new['entity:sample_id'] != sample_id]

Entry for sample 01dbf9fd8e117aff027799934b5598c3 has been modified.  Keeping changes.
Entry for sample 186085d11de539135e1df88895d0cb40 has been modified.  Keeping changes.
Entry for sample 22956e3990f48e610b82c32fccb94065 has been modified.  Keeping changes.
Entry for sample 22b126d950e0db336d7952664e968e68 has been modified.  Keeping changes.
Entry for sample 26a63c3c5d78bfbfe18bb611a975f092 has been modified.  Keeping changes.
Entry for sample 2ae6e552613be10b0569285e47ed7296 has been modified.  Keeping changes.
Entry for sample 318f21298c223c6e317b6d66c376784b has been modified.  Keeping changes.
Entry for sample 43ab67e9e5c7346162d2a0ecea50ebad has been modified.  Keeping changes.
Entry for sample 44012709b2afabddf710cf459d399eb5 has been modified.  Keeping changes.
Entry for sample 47963a34958914c6b49aed66dc8c07d3 has been modified.  Keeping changes.
Entry for sample 565743e25e0933b2492bac8ffcc86921 has been modified.  Keeping changes.
Entry for sample 5ce4d7b0da36bca8d08d0bbc3f

In [10]:
merged_tbl = pd.merge(tbl_old, tbl_new, how='outer') if len(ent_old) > 0 else tbl_new
merged_tbl = merged_tbl[['entity:sample_id'] + tbl_header]
merged_tbl = merged_tbl.drop_duplicates(subset=['flow_cell_id', 'instrument', 'sample_name', 'acquisition_stopped', 'processing_stopped', 'fast5_files_in_fallback', 'fast5_files_in_final_dest', 'fastq_files_in_fallback', 'fastq_files_in_final_dest'])
merged_tbl = merged_tbl[merged_tbl.fast5_files_in_final_dest != "0"]

In [13]:
merged_tbl

,entity:sample_id,final_summary_file,sequencing_summary_file,protocol_group_id,instrument,position,flow_cell_id,sample_name,basecalling_enabled,started,acquisition_stopped,processing_stopped,fast5_files_in_fallback,fast5_files_in_final_dest,fastq_files_in_fallback,fastq_files_in_final_dest
0,0047ded7aa70c3af8ae8db74af80ba97,gs://broad-gp-oxfordnano/Talkowski/SM-HB8DX/20...,gs://broad-gp-oxfordnano/Talkowski/SM-HB8DX/20...,Talkowski,PC48A003,2-A4-D4,PAE19301,SM-HB8DX,1,2019-11-25T18:27:45.392810+00:00,2019-11-28T18:27:56.978712+00:00,2019-11-28T18:27:56.978800+00:00,0,2248,0,2248
1,00822f3f511ad0ed5e19385f52414235,gs://broad-gp-oxfordnano/Talkowski/SM-HB8DU/20...,gs://broad-gp-oxfordnano/Talkowski/SM-HB8DU/20...,Talkowski,PC48A003,2-E1-H1,PAE08001,SM-HB8DU,1,2019-10-28T17:04:42.289671+00:00,2019-10-31T09:04:53.994249+00:00,2019-10-31T09:04:53.994348+00:00,0,2672,0,2672
2,010239e45eb252862c93432a697379ca,gs://broad-gp-oxfordnano/eQTL/2/20191113_1658_...,gs://broad-gp-oxfordnano/eQTL/2/20191113_1658_...,eQTL,PC48A003,1-A9-D9,PAE06444,2,1,2019-11-13T16:59:23.313164+00:00,2019-11-16T08:59:39.149164+00:00,2019-11-16T08:59:39.149266+00:00,0,5641,0,5603
3,01dbf9fd8e117aff027799934b5598c3,gs://broad-gp-oxfordnano/Talkowski/SM-HB8DW_RW...,gs://broad-gp-oxfordnano/Talkowski/SM-HB8DW_RW...,Talkowski,PC48A003,2-A3-D3,PAE18531,SM-HB8DW,1,2019-11-25T17:00:50.392312+00:00,2019-11-28T17:00:59.486151+00:00,2019-11-28T17:01:03.017732+00:00,0,1657,0,1657
4,02588f538e4ddb1bfe0fbc155240aff9,gs://broad-gp-oxfordnano/GMKF_Pilot/SM-I5Z7N/2...,gs://broad-gp-oxfordnano/GMKF_Pilot/SM-I5Z7N/2...,GMKF_Pilot,PC48A003,1C,PAG56613,SM-I5Z7N,1,2021-02-10T15:59:27.784478-05:00,2021-02-13T16:00:52.331312-05:00,2021-02-13T16:00:56.942162-05:00,0,4147,0,4147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,f9ae81ba59b9f69cd3f17c1aec3e7d8b,gs://broad-dsde-methods-long-reads-deepseq/ds7...,gs://broad-dsde-methods-long-reads-deepseq/ds7...,ds783,PCT0011,2-A11-D11,PAE09121,HG02982,1,2019-10-29T17:27:27.294506+00:00,2019-10-29T17:27:49.090037+00:00,2019-10-29T17:27:52.677389+00:00,0,1,0,1
121,fcab11344688872270106c6e2bfca583,gs://broad-gp-oxfordnano/Talkowski/SM-J1Y5L/20...,gs://broad-gp-oxfordnano/Talkowski/SM-J1Y5L/20...,Talkowski,PC48A003,2-E7-H7,PAE06567,SM-J1Y5L,1,2019-10-28T17:05:39.400663+00:00,2019-10-31T09:05:48.590202+00:00,2019-10-31T09:05:48.590271+00:00,0,2203,0,2203
122,fd9f500d6d0590f072b14f1328084255,gs://broad-gp-oxfordnano/Anne/SM-G5W8T/2019120...,gs://broad-gp-oxfordnano/Anne/SM-G5W8T/2019120...,Anne,PC48A003,2-A10-D10,PAE16552,SM-G5W8T,1,2019-12-05T14:10:00.107916+00:00,2019-12-05T16:25:50.697527+00:00,2019-12-05T16:25:50.697608+00:00,0,2,0,2
123,fdebded00238522740a5ff79a5a4f3f3,gs://broad-gp-oxfordnano/Talkowski/SM-HB8DW/20...,gs://broad-gp-oxfordnano/Talkowski/SM-HB8DW/20...,Talkowski,PC48A003,1-A1-D1,PAE08120,SM-HB8DW,1,2019-11-04T19:46:30.891247+00:00,2019-11-06T19:56:57.426217+00:00,2019-11-06T19:56:57.426287+00:00,0,592,0,592


## Upload new sample and sample_set tables to Terra

Upload the merged 'sample' table and sample_set table to this workspace.

In [27]:
upload_data(namespace, workspace, merged_tbl)

Uploaded 124 rows successfully.
Uploaded 77 sample sets successfully.
Uploaded 124 sample set members successfully.
